# M031(first half) and M025

In [2]:
import pandas as pd
import numpy as np

In [27]:
sub1 = pd.read_csv('../submissions/M025_0629_1311_submission_lgb_2folds_-1.3265CV_500000iter_0.1lr.csv')
sub2a = pd.read_csv('../temp/temp4of8_M030_0701_1300_submission_lgb_3folds_5000000iter_0.05lr.csv')
sub2b = pd.read_csv('../temp/temp4of8_M030_0701_1302_submission_lgb_3folds_5000000iter_0.05lr.csv')
sub2c = pd.read_csv('../temp/temp4of8_M030_0701_1304_submission_lgb_3folds_5000000iter_0.05lr.csv')

In [7]:
sub2 = sub1.copy()

In [9]:
sub2['scc_a'] = sub2a['scalar_coupling_constant']
sub2['scc_b'] = sub2b['scalar_coupling_constant']
sub2['scc_c'] = sub2c['scalar_coupling_constant']

In [12]:
sub2['scalar_coupling_constant'] = sub2[['scc_a','scc_b','scc_c']].mean(axis=1)

In [21]:
sub3 = sub2.drop(['scc_a','scc_b','scc_c'], axis=1).copy()

In [22]:
sub3.loc[sub2['scalar_coupling_constant'] == 0, 'scalar_coupling_constant'] = sub1.loc[sub2['scalar_coupling_constant'] == 0]

In [23]:
np.corrcoef(sub3['scalar_coupling_constant'], sub2['scalar_coupling_constant'])

array([[1.        , 0.99455296],
       [0.99455296, 1.        ]])

In [24]:
np.corrcoef(sub3['scalar_coupling_constant'], sub1['scalar_coupling_constant'])

array([[1.        , 0.99997242],
       [0.99997242, 1.        ]])

In [25]:
sub3.head()

,id,scalar_coupling_constant
0,4658147,8.873624
1,4658148,178.323424
2,4658149,3.716820
3,4658150,178.323424
4,4658151,8.873624


In [26]:
(sub3['scalar_coupling_constant'] == 0).sum()

0

In [28]:
oof2a = pd.read_csv('../temp/temp4of8_M030_0701_1300_oof_lgb_3folds_5000000iter_0.05lr.csv')
oof2b = pd.read_csv('../temp/temp4of8_M030_0701_1302_oof_lgb_3folds_5000000iter_0.05lr.csv')
oof2c = pd.read_csv('../temp/temp4of8_M030_0701_1304_oof_lgb_3folds_5000000iter_0.05lr.csv')

In [29]:
oof1 = pd.read_csv('../oof/M025_0629_1311_oof_lgb_2folds_-1.3265CV_500000iter_0.1lr.csv')
oof2 = oof1.copy()

In [32]:
oof2['oof_a'] = oof2a['oof_preds']
oof2['oof_b'] = oof2b['oof_preds']
oof2['oof_c'] = oof2c['oof_preds']

In [38]:
oof2['oof_preds'] = oof2[['oof_a','oof_b','oof_c']].sum(axis=1)
oof3 = oof2.drop(['oof_a','oof_b','oof_c'], axis=1).copy()

In [39]:
oof3.loc[oof2['oof_preds'] == 0, 'oof_preds'] = oof1.loc[oof2['oof_preds'] == 0]

In [40]:
oof3.head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,83.769148
1,1,2JHH,-11.2570,-10.637787
2,2,2JHH,-11.2548,-10.637787
3,3,2JHH,-11.2543,-10.631721
4,4,1JHC,84.8074,83.769148


## OOF Scores

In [41]:
from sklearn.metrics import mean_absolute_error

for i, d in oof3.groupby('type'):
    print(i)
    print(mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds']))

1JHC
0.6206077451007648
1JHN
0.35815790678334736
2JHC
0.30022325062851835
2JHH
0.16464835967967395
2JHN
0.139645352363473
3JHC
0.39051763625418157
3JHH
0.1934145785886842
3JHN
0.1627272677520273


In [42]:
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [43]:
group_mean_log_mae(oof3['scalar_coupling_constant'], oof3['oof_preds'], oof3['type'])

-1.3598174898105542

In [44]:
sub3.to_csv('../submissions/MERGE004-1J2J-M025-3J-M030_-1.3598CV.csv', index=False)